In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
from scipy import io
from scipy import stats
import pickle
import glob
import pandas as pd
import math

#for mat file 
import scipy.io as sio
import seaborn as sb

# import user defined functions
from functions import *
from utils import * 

# Import decoder functions
from decoders import LSTMDecoder


from itertools import product

2023-12-08 11:03:35.934339: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
def make_mat_file(res_all,result_file):

    #save decoding result in .mat format
    mat_name='./mat_data/{}'.format(result_file.split('/')[-2])
    make_dir(mat_name)
    for k in range(len(res_all)):
        temp=res_all[k]
        mn=temp['model']
        dn=temp['data_name']
        save_name='{}/{}_{}.mat'.format(mat_name,dn,mn)
        sio.savemat(save_name,temp)

In [3]:
def model_eval_not_flat(model):
    
    #train
    model.fit(X_train, y_train)
    #prediction
    y_test_predicted=model.predict(X_test)
    y_test_predicted=y_test_predicted * y_train_std + y_train_mean
    #evaluation
    corr,R2,mae,y_test_predicted_smooth,corr_smooth,R2_smooth,mae_smooth=calc(y_test,y_test_predicted)
    #plot    
    plot_all(y_test,y_test_predicted,out,data_name,model_name,target_name)
        
    res={'data_name':data_name,
         'y_test':y_test,
         'y_test_predicted':y_test_predicted,
         'corr':corr,
         'R2':R2,
         'mae':mae,
         'y_test_predicted_smooth':y_test_predicted_smooth,
         'corr_smooth':corr_smooth,
         'R2_smooth':R2_smooth,
         'mae_smooth':mae_smooth,
         'model':model_name,
         'n_neuron':n_neuron}

    return res

In [4]:
def velocity2(position):
    lenposi=len(position)
    v_cm_s=[]
    actual_posi=[]
    v_m=[];
    actual_posi=position*(40.0/200.0)
    for k in range(lenposi):
        tmp=0
        
        for i in range(-6,5): #515 filter
            if (k+i <= 0 or k+i+1 >= lenposi):
                tmp=tmp
            else:
                xaya = actual_posi[k+i+1] - actual_posi[k+i]             
                dist = np.sqrt(np.sum(xaya**2))
                tmp=tmp+dist
        v_cm_s.append(tmp*(10.0/11.0)) #515 filter
    v_m=np.array(v_cm_s)
    
    return  v_m

In [ ]:
velocity_th_list=[0]


for th_velo, info_ind in product(velocity_th_list, [0]):
    out='./Sp_vf_{}'.format(th_velo)
        
    make_dir(out)
    result_dir=out#dir of decoding result         

    result_file='{}/res_all.pkl'.format(result_dir)    
    data_dir='./data/'
    
    #choose data you want to use for decoding
    datalist_all=glob.glob('{}/*'.format(data_dir))
    
    
    
    #######################
    np.random.seed(100)

    res_all=[]
    pf_score_all=[]
    for datalist_name in datalist_all:
        print(datalist_name)
        datalist=glob.glob(datalist_name + '/*')
        data_name=datalist[0].split('/')[-2]

        #load data
        trace,spike,position,x_position,y_position,n_neuron,n_time,firing_rate,trace_smooth=data_load(datalist)      
         

        #thresholding by velocity
        use_time_v=v_th(position,th_velo)
       
        trace=trace[use_time_v,:]
        spike=spike[use_time_v,:]
        position=position[use_time_v,:]
        x_position=x_position[use_time_v]
        y_position=y_position[use_time_v]
        n_time=np.sum(use_time_v)
        firing_rate=firing_rate[use_time_v,:]
        trace_smooth=trace_smooth[use_time_v,:]
        
        ## Set what part of data should be part of the training/testing sets
        train_range=[0, 0.5]
        test_range=[0.5, 1.0]

        train_set=np.arange(np.int(np.round(train_range[0]*n_time)),   np.int(np.round(train_range[1]*n_time)))
        test_set=np.arange(np.int(np.round(test_range[0]*n_time)),   np.int(np.round(test_range[1]*n_time)))
            
        use_data=np.sum(spike[train_set,:],axis=0)>0#remove the neuron which dont activate at training
        spike=spike[:,use_data]
        trace=trace[:,use_data]
        firing_rate=firing_rate[:,use_data]
        trace_smooth=trace_smooth[:,use_data]
        
        n_neuron=spike.shape[1]
        
        
        ########################

        
        firing_rate=firing_rate-np.mean(firing_rate,axis=0,keepdims=True)      

        X=get_spikes_with_history(firing_rate, 0, 0, 1)
        X_flat=X.reshape(X.shape[0],(X.shape[1]*X.shape[2]))        
        

        ########################
                
        target_name = ['speed'] #target variable name for plot
        y=velocity2(position).reshape(-1,1) #target variable. shape is (n_time , 1)        
        

        ### 3C. Split into training / testing / testation sets

        #Get training data
        X_train=X[train_set,:,:]
        X_flat_train=X_flat[train_set,:]
        y_train=y[train_set,:]

        #Get testing data
        X_test=X[test_set,:,:]
        X_flat_test=X_flat[test_set,:]
        y_test=y[test_set,:]

        #Z-score "X" inputs. 
        X_train_mean=np.nanmean(X_train,axis=0)
        X_train_std=np.nanstd(X_train,axis=0)
        X_train=(X_train-X_train_mean)/X_train_std
        X_test=(X_test-X_train_mean)/X_train_std

        #Z-score "X_flat" inputs. 
        X_flat_train_mean=np.nanmean(X_flat_train,axis=0)
        X_flat_train_std=np.nanstd(X_flat_train,axis=0)
        X_flat_train=(X_flat_train-X_flat_train_mean)/X_flat_train_std
        X_flat_test=(X_flat_test-X_flat_train_mean)/X_flat_train_std

        #Zero-center or zscore outputs
        y_train_mean=np.mean(y_train,axis=0)
        y_train_std=np.std(y_train,axis=0)
        y_train_std_svr=np.std(y_train,axis=0)

        y_train_std=y_train_std*0 +1
        y_train_svr=(y_train-y_train_mean)/y_train_std_svr
        y_train=(y_train-y_train_mean)/y_train_std

        
        ## 4. Run Decoders-----------------------------------------------------------------------
            
        # hyper parameters, for neural networks
        epoch=10
        unit=1000
        dropout=0.1

        model_name='lstm'
        model=LSTMDecoder(units=unit,dropout=dropout,num_epochs=epoch)
        res=model_eval_not_flat(model)
        res_all.append(res)
        print('{} : corr : {}'.format(model_name,res['corr_smooth'])) 
        

        plt.close('all')
        print('----')
        
 

    with open('{}/res_all.pkl'.format(out),'wb') as f:
        pickle.dump(res_all,f)

    result={'dropout':dropout,
            'unit':unit}
    

    with open('{}/parameters.pkl'.format(out),'wb') as f:
        pickle.dump(result,f)
        
    ##Generate mat file
    make_mat_file(res_all,result_file)